# Research Question
For our project, we will be researching crime data in Los Angeles and how that data is affected by various variables such as COVID-19, educational attainment, and household income. Due to the global pandemic, crime rates have fluctuated substantially due to the lockdown and the reopening of the county.

# Data Sources
- Crime Data from 2020 to present, https://data.lacity.org/A-Safe-City/Crime-Data-from-2020-to-Present/2nrs-mtv8
- COVID-19 Data from 2020 to present, https://github.com/datadesk/california-coronavirus-data/blob/master/latimes-place-totals.csv
- Educational Attainment for LA County (2014-2018), Social Explorer
- Household Income for LA County (2018), Social Explorer
- White vs. Non-White Homeowners (2018), Social Explorer
- Mapping Inequality/ Home Owners Loan Corporation (HOLC) LA Redlining Map (1939), clsl.richmond.edu 

# Data Exploration and Analysis

Now we want to explore our data sources and provide an analysis of our datasets.

## COVID-19 Rates in California
We will begin our data exploration by importing the current COVID-19 data from the LA times.

In [ ]:
import plotly.express as px
import pandas as pd

In [ ]:
latimes = pd.read_csv(
    "https://raw.githubusercontent.com/datadesk/california-coronavirus-data/master/latimes-place-totals.csv")

In [ ]:
# Now we want to get some basic statistics from the dataset. How many rows and columns?
latimes.shape

In [ ]:
#What are the first 5 rows?
latimes.head()

In [ ]:
# dataframe info?
latimes.info()

In [ ]:
# Next, we want to clean up the data. This includes empty coordinates, empty confirmed cases, and incorrect coordinates (Note: positive longitudes do not exist in California)
# We do this by using the .query() method that allows us to query and filter the dataset using SQL syntax.
latimes.query("confirmed_cases == 'NaN'")

In [ ]:
# NaN values for 'x'?
latimes.query("x == 'NaN'")

In [ ]:
#NaN values for 'y'?
latimes.query("y == 'NaN'")

In [ ]:
# Positive longitude coordinates?
latimes.query("x > 0")

In [ ]:
# Do we have any null dates?
latimes.query("date.isnull()", engine='python')

In [ ]:
# Now we will combine our arguments and clean the data:
latimes = latimes.query("confirmed_cases != 'NaN' & x < 0 & x != 'NaN' & date.notnull()", engine='python')
latimes.head()

In [ ]:
# How many records do we have now?
latimes.shape
# Less columns than before

In [ ]:
# Now we want to look at more statistics in our dataset. Let's look at confirmed cases.
latimes.confirmed_cases.describe()

In [ ]:
# Since our research question is focused on Los Angeles County, let's look at which cities in LA County have the highest confirmed cases. 
latimes_LA = latimes.query("county=='Los Angeles'")

In [ ]:
latimes_LA.groupby("place").confirmed_cases.describe().sort_values(by=["max"], ascending=False).head(50)

In [ ]:
# Let's create a bar chart representing the confirmed cases in LA County overtime.
LACounty = latimes.query("county == ['Los Angeles']")
px.bar(LACounty,
      x='date',
      y='confirmed_cases')

In [ ]:
# Let's be more specific. Let's create a bar chart of the top three cities in LA County with the highest confirmed cases: Long Beach, East Los Angeles, and Pomona.
TopLA = latimes.query("place == ['Long Beach','East Los Angeles','Pomona']")
px.bar(TopLA,
      x='date',
      y='confirmed_cases',
      color = 'place')

Now that we've looked at the top three cities with the highest confirmed COVID-19 cases, let's represent our dataset in a different visualization format. 
Let's create an animated scatter plot to represent the change overtime of confirmed cases in cities across LA County.

In [ ]:
# What is the mean of  confirmed cases in LA County?
latimes_LA_mean = latimes_LA.confirmed_cases.mean()
latimes_LA_mean

In [ ]:
px.scatter(latimes_LA,
           x='x',
           y='y',
           color='confirmed_cases', 
           size='confirmed_cases',
           size_max=40, 
           hover_name='place',
           animation_frame='date', # this creates a frame by frame animation by day
           color_continuous_scale = 'RdYlGn_r',
           range_color = (0,latimes_LA_mean*2))

An issue we had with this animated scatterplot was that the animation began from the present to March, instead of vice versa. This is something we will have to address in the future.

## Crime Rates in the City of Los Angeles
Let's look at crime rates in LA County from 2020 to present. We will begin by importing the data.

In [ ]:
import pandas as pd
import plotly.express as px
from sodapy import Socrata

Next, we acquire the data using the socrata API. 
- https://dev.socrata.com/foundry/data.lacity.org/2nrs-mtv8

In [ ]:
# connect to the data portal
client = Socrata("data.lacity.org", None)

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("2nrs-mtv8", limit=2000)

# Convert to pandas DataFrame
df = pd.DataFrame.from_records(results)

# print it with .sample, which gives you random rows
df.sample(2)

In [ ]:
# Now, we want to add a "where" statement to look at the data from March 1, 2020 to April 30, 2020, limited to 30,000.
results = client.get("2nrs-mtv8", 
                     limit = 30000, 
                     where = "date_rptd between '2020-03-01T00:00:00' and '2020-04-30T00:00:00'"
                    )

In [ ]:
# Convert to pandas DataFrame
df = pd.DataFrame.from_records(results)

### Data Exploration and Analysis of Crime Data

In [ ]:
# how many rows and columns?
df.shape

In [ ]:
# what fields and datatypes?
df.info()

In [ ]:
# First 5 rows?
df.head()

In [ ]:
# Let's create a bar graph and add labels.
px.bar(df,
       x='date_rptd',
       title='Crime Rates in Los Angeles, March to April 2020',
       labels={'date_rptd':'Date of Crimes','count':'Number of Crimes'}
      )

In [ ]:
# Let's look at the distinct value of charges
df.crm_cd_desc.unique()

In [ ]:
# Let's look at  the top 25 distinct value of charges
crime_by_type = df.crm_cd_desc.value_counts().reset_index()
crime_by_type.head(25)

In [ ]:
# Rename our columns
crime_by_type.columns=['crime','count']
crime_by_type.head(25)

In [ ]:
# Let's create a bar chart to represent the Top 25 Crimes.
px.bar(crime_by_type.head(25).sort_values(by='count',ascending=True),
       y='crime',
       x='count',
       orientation= 'h',
       title='Crime Rates in Los Angeles, March to April 2020')

In [ ]:
# Now, let's subset our data and begin mapping the dataset.
df.info()

Let's eliminate the unnecessary fields and create a subset of the data with just the following fields:

- `date_rptd`
- `crm_cd`
- `crm_cd_desc`
- `lat`
- `lon`

In [ ]:
# subset the data
df_mini = df[['date_rptd','crm_cd','crm_cd_desc','lat','lon']].copy()
df_mini.head()

In [ ]:
# Check the info for our subset data
df_mini.info()

In [ ]:
# Now we want to convert latitude and longitude to floats
df_mini['lat'] = df_mini['lat'].astype(float)
df_mini['lon'] = df_mini['lon'].astype(float)
df_mini.info()

In [ ]:
# What if we try to map it with plotly?
fig = px.scatter_mapbox(df_mini,
                        lat='lat',
                        lon='lon',
                        mapbox_style="stamen-terrain")
fig.show()

# Group Contributions
1. Donna Heydar (Donna contributed to breaking down the educational attainment data in Los Angeles County. Both members discussed which datasets to use and discussed similarities between the two after breaking them down. Donna also contributed to the data exploration and analysis of Crime data in LA as well as COVID-19 data,)
2. Daniel Ruiz (Daniel contributed to breaking down the household income data in Los Angeles County. Both members discussed which datasets to use and discussed similarities between the two after breaking them down. Daniel also contributed to the data exploration and analysis of HOLC Redlining. He also contributed to the comparison between education  income, and homeownership in Los Angeles.)